In [16]:
import os
import glob
import numpy as np
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import gradio as gr
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

import json

Stage 1 -> Data ingestion

In [17]:
#fetching the dataset
files = glob.glob(
    r"C:\Users\hitan\.cache\huggingface\hub\datasets--next-tat--TAT-QA\**\*.json",
    recursive=True
)

files

['C:\\Users\\hitan\\.cache\\huggingface\\hub\\datasets--next-tat--TAT-QA\\snapshots\\c96247f5077eac447f63527fd3dcfdc58bb56d6a\\tatqa_dataset_dev.json',
 'C:\\Users\\hitan\\.cache\\huggingface\\hub\\datasets--next-tat--TAT-QA\\snapshots\\c96247f5077eac447f63527fd3dcfdc58bb56d6a\\tatqa_dataset_test.json',
 'C:\\Users\\hitan\\.cache\\huggingface\\hub\\datasets--next-tat--TAT-QA\\snapshots\\c96247f5077eac447f63527fd3dcfdc58bb56d6a\\tatqa_dataset_test_gold.json',
 'C:\\Users\\hitan\\.cache\\huggingface\\hub\\datasets--next-tat--TAT-QA\\snapshots\\c96247f5077eac447f63527fd3dcfdc58bb56d6a\\tatqa_dataset_train.json']

In [18]:
print({len(files)})

{4}


In [19]:
with open(files[3], "r", encoding = "utf-8") as f:
    data = json.load(f)

len(data), data[0].keys()

(2207, dict_keys(['table', 'paragraphs', 'questions']))

Stage 2 -> Load LLM

In [20]:
#loading the llama model by using huggingface pipelines
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

Llama = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(Llama)
model = AutoModelForCausalLM.from_pretrained(
    Llama,
    dtype = torch.float16,
    device_map = "auto"
)

model.config.pad_token_id = model.config.eos_token_id

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,  ####### 512->256
    temperature=0.1,      ####### 0.7->0.3->0.1 more precise outputs
    do_sample = True,
    top_p = 0.9   
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.36s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0
